In [2]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

df = sqlContext.read.load('file:///home/cloudera/Downloads/big-data-4/daily_weather.csv', format='com.databricks.spark.csv',header='true',inferSchema='true')


In [3]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
number,1095,547.0,316.0991405661627,0,1094
air_pressure_9am,1092,918.8825513138094,3.182702897404423,907.9900000000024,929.3200000000012
air_temp_9am,1090,64.93300141287072,11.170386444290106,36.752000000000685,98.90599999999992
avg_wind_direction_9am,1091,142.2355107005759,69.10616647985104,15.500000000000046,343.4
avg_wind_speed_9am,1092,5.50828424225493,4.550728366514399,0.69345139999974,23.554978199999763
max_wind_direction_9am,1092,148.95351796516923,67.20721925861356,28.89999999999991,312.19999999999993
max_wind_speed_9am,1091,7.019513529175272,5.595642950535739,1.1855782000000479,29.84077959999996
rain_accumulation_9am,1089,0.20307895225211126,1.593220115157986,0.0,24.01999999999907
rain_duration_9am,1092,294.1080522756142,1597.3468900123448,0.0,17704.0


In [4]:
df.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141287072|
| stddev|11.170386444290106|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [5]:
df.count()

1095

In [6]:
removeAllDF = df.na.drop()

In [7]:
removeAllDF.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1064|
|   mean| 65.02260949558733|
| stddev|11.162784079702057|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [8]:
removeAllDF.count()

1064

In [9]:
from pyspark.sql.functions import avg
imputeDF = df

In [10]:
for x in imputeDF.columns:
    meanValue = removeAllDF.agg(avg(x)).first()[0]
    print(x, meanValue)
    imputeDF = imputeDF.na.fill(meanValue, [x])

number 545.0018796992481
air_pressure_9am 918.9031798641051
air_temp_9am 65.02260949558733
avg_wind_direction_9am 142.30675564934037
avg_wind_speed_9am 5.48579305071369
max_wind_direction_9am 148.48042413321315
max_wind_speed_9am 6.999713658875691
rain_accumulation_9am 0.18202347650615522
rain_duration_9am 266.3936973996037
relative_humidity_9am 34.07743985327709
relative_humidity_3pm 35.14838093290533


In [11]:
df.describe(['air_temp_9am']).show()
imputeDF.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141287072|
| stddev|11.170386444290106|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1095|
|   mean| 64.93341058219818|
| stddev| 11.14485573644398|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [34]:
removeAllDF.select(['air_pressure_9am']).filter(removeAllDF.air_pressure_9am > 911.736).filter(removeAllDF.air_pressure_9am < 914.67).count()

77

In [25]:
from pyspark.sql.functions import min
imputeMinDF = df

In [26]:
for x in imputeMinDF.columns:
    minValue = removeAllDF.agg(min(x)).first()[0]
    print(x, minValue)
    imputeMinDF = imputeMinDF.na.fill(minValue, [x])

number 0
air_pressure_9am 907.9900000000024
air_temp_9am 36.752000000000685
avg_wind_direction_9am 15.500000000000046
avg_wind_speed_9am 0.69345139999974
max_wind_direction_9am 28.89999999999991
max_wind_speed_9am 1.1855782000000479
rain_accumulation_9am 0.0
rain_duration_9am 0.0
relative_humidity_9am 6.090000000001012
relative_humidity_3pm 5.3000000000006855


In [29]:
imputeMinDF.select(['air_temp_9am']).filter(imputeMinDF.air_temp_9am < 42.292).count()

28

In [31]:
from pyspark.sql.functions import col
df.select(['air_pressure_9am']).filter(col('air_pressure_9am').isNull()).show()

+----------------+
|air_pressure_9am|
+----------------+
|            null|
|            null|
|            null|
+----------------+



In [32]:
df.select(['rain_accumulation_9am']).filter(col('rain_accumulation_9am').isNull()).show()

+---------------------+
|rain_accumulation_9am|
+---------------------+
|                 null|
|                 null|
|                 null|
|                 null|
|                 null|
|                 null|
+---------------------+



In [33]:
df.select(['air_temp_9am']).filter(col('air_temp_9am').isNull()).show()

+------------+
|air_temp_9am|
+------------+
|        null|
|        null|
|        null|
|        null|
|        null|
+------------+

